# Dash Exercise: basketball player stat

In [ ]:
#!pip install dash jupyter_dash
#!pip install --upgrade plotly

In [19]:
from jupyter_dash import JupyterDash # Dash jupyter inline
import dash_html_components as html 
import dash_core_components as dcc 
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import numpy as np

### simple callback example

In [1]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H5("Change the value in the text box to see callbacks in action!"),
    html.Div(["Input: ",
              dcc.Input(id='my-input', value='initial value', type='text')]),
    html.Br(),
    html.Div(id='my-output'),

])


@app.callback(
    Output(component_id='my-output', component_property='children'),
    [Input(component_id='my-input', component_property='value')]
)
def update_output_div(input_value):
    return 'Output: {}'.format(input_value)

# parameter mode=inline, run_server will start on jupyter notebook
app.run_server(mode='inline')

NameError: name 'JupyterDash' is not defined

In [16]:
# readin csv file into dataframe, Plotly mainly access Ddataframe as its input data format
df = pd.read_csv('../data/players_stats_by_season_full_details.csv')
df.head()

League       Season           Stage         Player Team  GP     MIN  FGM  \
0    NBA  2009 - 2010  Regular_Season   Kevin Durant  OKC  82  3239.3  794   
1    NBA  2009 - 2010  Regular_Season   LeBron James  CLE  76  2965.6  768   
2    NBA  2009 - 2010  Regular_Season    Dwyane Wade  MIA  77  2792.4  719   
3    NBA  2009 - 2010  Regular_Season  Dirk Nowitzki  DAL  81  3038.8  720   
4    NBA  2009 - 2010  Regular_Season    Kobe Bryant  LAL  73  2835.4  716   

    FGA  3PM  ...   PTS  birth_year  birth_month    birth_date  height  \
0  1668  128  ...  2472      1988.0          Sep  Sep 29, 1988     6-9   
1  1528  129  ...  2258      1984.0          Dec  Dec 30, 1984     6-8   
2  1511   73  ...  2045      1982.0          Jan  Jan 17, 1982     6-4   
3  1496   51  ...  2027      1978.0          Jun  Jun 19, 1978     7-0   
4  1569   99  ...  1970      1978.0          Aug  Aug 23, 1978     6-6   

   height_cm  weight  weight_kg    nationality  \
0      206.0   240.0      109.0  United States   
1      203.0   250.0      113.0  United States   
2      193.0   220.0      100.0  United States   
3      213.0   245.0      111.0        Germany   
4      198.0   212.0       96.0  United States   

                        high_school  
0         Montrose Christian School  
1  St. Vincent St. Mary High School  
2    Harold L. Richards High School  
3                               NaN  
4          Lower Merion High School  

[5 rows x 31 columns]

In [17]:
# summing up for each player
df=df.groupby(["Player","League"]).sum().drop(["birth_year","height_cm","weight","weight_kg"],axis=1)
features = df.columns.to_list()
df.reset_index(inplace=True)
df.head()

Player             League  GP     MIN  FGM  FGA  3PM  3PA  FTM  \
0  A'uston Calhoun      Bulgarian-NBL  26   528.0   96  214   14   46   31   
1  A'uston Calhoun          Swiss-LNA  24   832.0  150  344   32   95   65   
2    A.D. Vassallo  French-Jeep-Elite  32  1011.0  192  423   90  211   99   
3       A.J. Davis       Canadian-NBL  35   814.8  132  293   25   98   71   
4    A.J. Gasporra       Mexican-LNBP  33   500.3   69  147   34   78   42   

   FTA  TOV  PF  ORB  DRB  REB  AST  STL  BLK  PTS  
0   50   31  58   49   66  115   25   11    6  237  
1   78   63  40   48  112  160   45   19   13  397  
2  121   89  85   24  113  137   87   37    9  573  
3   94   89  78   19  121  140   83   26    8  360  
4   49   25  54   22   47   69   50   21    1  214

In [23]:
# Make a dropdown to select which "stat" parameter to draw histogram
# Make a dropdown to select which "league" to draw histogram
# graph is histogram of stat feature
app = JupyterDash(__name__)

app.layout = html.Div([
                       html.Div([
                                 dcc.Dropdown(
                                     id='feature_selector',
                                     options=[{'value': cnt, 'label': cnt} for cnt in features],
                                     value='3PA',
                                     multi=False,
                                     style={'width': '50%', 'display': 'inline-block'}
                                 ),
                                 dcc.Dropdown(
                                     id='league_selector',
                                     options=[{'value': fac, 'label': fac} for fac in df.League.unique()],
                                     value=['NBA'],
                                     multi=True,
                                     style={'width': '50%', 'display': 'inline-block'}
                                 )
                       ]),
                       dcc.Graph(id='basketball_graph')
])

@app.callback(Output('basketball_graph', 'figure'), [Input('feature_selector', 'value'), Input('league_selector', 'value')])
def histogram_graph(selected_feature, selected_league):
    selected_league = df[df['League'].isin(selected_league)]
    figure = px.histogram(selected_league, x=selected_feature, color='League')
    figure = figure.update_layout(barmode="group")
    return figure

# parameter mode=inline, run_server will start on jupyter notebook
app.run_server(mode='inline')